# GRU and LSTM
* In this demo, we will apply the GRU and LSTM on the Sunspot dataset (https://www.kaggle.com/robervalt/sunspots)
* See demo in Lecture 18 for details about dataset and pre-processing.

## 1. Read, pre-process, and split dataset

In [ ]:
import numpy as np
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler

# Download and read dataset
sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
df = read_csv(sunspots_url)

# Pre-process features
scaler = MinMaxScaler(feature_range=(0, 1))
data = np.array(df['Sunspots'], dtype='float32').reshape((-1, 1))
data = scaler.fit_transform(data).flatten()

# Split dataset into train/test sets
split_ratio = 0.8 # Percentage to use for training

split_index = int(len(data)*split_ratio) # Index to split train/test sets
train_data = data[:split_index]
test_data = data[split_index:]

print(train_data.shape, test_data.shape)

## 2. Prepare inputs and targets for each split

In [ ]:
# Function to prepare the input X and target Y
def get_XY(dat, time_steps, sliding_steps):
    # Prepare the targets Y
    Y_ind = np.arange(time_steps, len(dat), sliding_steps)
    Y = dat[Y_ind]
    
    # Prepare the inputs X
    X = []
    for i in Y_ind:
        X.append(dat[i-time_steps:i])
    X = np.array(X).reshape((len(Y), time_steps, 1))
    
    return X, Y

time_steps = 12
sliding_steps = 12
trainX, trainY = get_XY(train_data, time_steps, sliding_steps)
testX, testY = get_XY(test_data, time_steps, sliding_steps)

## 3. Define, train, and test GRU model
* We define an GRU model with 3 hidden nodes, then a Dense layer with 1 hidden node (for regression).
* We train and evaluate the model using MSE loss.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, GRU
from sklearn.metrics import mean_squared_error

# Define model
model_gru = Sequential()
model_gru.add(GRU(units=3, input_shape=(time_steps, 1)))
model_gru.add(Dense(units=1, activation='tanh'))

# Compile and train model
model_gru.compile(loss='mean_squared_error', optimizer='adam')
model_gru.fit(trainX, trainY, epochs=20, batch_size=5)

# Make prediction and compute test MSE
train_predict_gru = model_gru.predict(trainX)
test_predict_gru = model_gru.predict(testX)

train_mse = mean_squared_error(trainY, train_predict_gru)
test_mse = mean_squared_error(testY, test_predict_gru)

print('Train MSE: %.3f' % (train_mse))
print('Test MSE: %.3f' % (test_mse))

## 4. Define, train, and test LSTM model
* We define an LSTM model with 3 hidden nodes, then a Dense layer with 1 hidden node (for regression).
* We train and evaluate the model using MSE loss.

In [ ]:
from keras.layers import LSTM

# Define model
model_lstm = Sequential()
model_lstm.add(LSTM(units=3, input_shape=(time_steps, 1)))
model_lstm.add(Dense(units=1, activation='tanh'))

# Compile and train model
model_lstm.compile(loss='mean_squared_error', optimizer='adam')
model_lstm.fit(trainX, trainY, epochs=40, batch_size=5)

# Make prediction and compute test MSE
train_predict_lstm = model_lstm.predict(trainX)
test_predict_lstm = model_lstm.predict(testX)

train_mse = mean_squared_error(trainY, train_predict_lstm)
test_mse = mean_squared_error(testY, test_predict_lstm)

print('Train MSE: %.3f' % (train_mse))
print('Test MSE: %.3f' % (test_mse))

## 5. Plot the true targets and predictions

In [ ]:
import matplotlib.pyplot as plt

true_targets = np.append(trainY, testY)
predicted_targets_gru = np.append(train_predict_gru, test_predict_gru)
predicted_targets_lstm = np.append(train_predict_lstm, test_predict_lstm)

plt.figure(figsize=(15, 6), dpi=80)
plt.plot(true_targets)
plt.plot(predicted_targets_gru)
plt.plot(predicted_targets_lstm)
plt.axvline(x=len(trainY), color='r')
plt.legend(['True target', 'GRU Prediction', 'LSTM Prediction'])
plt.xlabel('Target index')
plt.ylabel('Sunspots scaled')
plt.title('Actual and predicted target values. The red line separates the training and test examples.')
plt.show()